In [56]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

In [84]:
preco = pd.read_csv('/home/wilmer/Documentos/Codes/tesis/Instancias/test/test1/preco.csv')
demanda = pd.read_csv('/home/wilmer/Documentos/Codes/tesis/Instancias/test/test1/demanda.csv')

In [85]:
# Definir los datos de demanda y precios
precios = [61, 46, 30]
demandas = [1, 7, 3]

# Calcular probabilidades
total_demanda = sum(demandas)
probabilidades = [d / total_demanda for d in demandas]

print("Probabilidades teóricas:", probabilidades)

# Número de simulaciones
num_simulaciones = 10000

# Generar la simulación con selección aleatoria ponderada
simulaciones = np.random.choice(precios, size=num_simulaciones, p=probabilidades)
print(simulaciones)

# Contar la frecuencia de cada precio en la simulación
resultados = [np.sum(simulaciones == precio) for precio in precios]
# resultados = [r / num_simulaciones for r in resultados]
print("Probabilidades simuladas:", resultados)


Probabilidades teóricas: [0.09090909090909091, 0.6363636363636364, 0.2727272727272727]
[30 46 46 ... 46 61 46]
Probabilidades simuladas: [np.int64(932), np.int64(6298), np.int64(2770)]


In [99]:
def mtr(fila, dem, pre, ns):

    merged_df = dem.merge(pre, on=['Origin', 'Destination', 'Vagon', 'Class'], how='left')
    
    class_dem = merged_df.loc[
        (merged_df['Origin']==fila.Origin) & 
        (merged_df['Destination']==fila.Destination) &
        (merged_df['Vagon']==fila.Vagon) &
        (merged_df['DBD']==fila.DBD)]
    
    if class_dem['Class'].shape[0] == 1:
        return 1
    else:
        # Definir los datos de demanda y precios
        precos = class_dem.Revenue.values
        demandas = class_dem.Bookings.values

        # Calcular probabilidades teoricas
        prob_teoricas = [d / demandas.sum() for d in demandas]

        # Generar la simulación con selección aleatoria ponderada
        simulaciones = np.random.choice(precos, size=ns, p=prob_teoricas)

        resultado = np.sum(simulaciones == class_dem.loc[class_dem['Class']==fila.Class].Revenue.values)/ns

        return resultado


In [100]:
demanda['MTR'] = demanda.apply(mtr, axis=1, pre = preco, dem = demanda, ns = 10000)
demanda

,Origin,Destination,Vagon,Class,DBD,Bookings,MTR
0,BAL,WAS,Z,8,1,1,1.0000
1,BWI,WAS,Z,6,59,1,1.0000
2,PHL,BAL,P,7,1,2,1.0000
3,PHL,BAL,Z,6,1,1,0.4954
4,PHL,BAL,Z,8,1,1,0.4994
5,PHL,BAL,Z,8,5,1,1.0000
6,PHL,BWI,Z,8,6,2,1.0000
7,PHL,WAS,P,6,1,1,1.0000
8,PHL,WAS,Z,5,0,1,1.0000
9,PHL,WAS,Z,6,1,1,0.0891


In [89]:
# Realizar el left join
merged_df = demanda.merge(preco, on=['Origin', 'Destination', 'Vagon', 'Class'], how='left')

# Mostrar el resultado
print(merged_df)

   Origin Destination Vagon  Class  DBD  Bookings  Revenue
0     BAL         WAS     Z      8    1         1     9.50
1     BWI         WAS     Z      6   59         1    18.00
2     PHL         BAL     P      7    1         2    65.00
3     PHL         BAL     Z      6    1         1    48.00
4     PHL         BAL     Z      8    1         1    24.00
5     PHL         BAL     Z      8    5         1    24.00
6     PHL         BWI     Z      8    6         2    24.00
7     PHL         WAS     P      6    1         1    99.00
8     PHL         WAS     Z      5    0         1    85.00
9     PHL         WAS     Z      6    1         1    61.00
10    PHL         WAS     Z      7    1         7    46.00
11    PHL         WAS     Z      8    1         3    30.00
12    PHL         WAS     Z      8    3         1    30.00
13    PHL         WAS     Z      8    8         1    30.00
14    PHL         WAS     Z      8   11         1    30.00
15    PHL         WAS     Z      8   15         1    30.

In [91]:
xx = merged_df.loc[
        (merged_df['Origin']=='PHL') & 
        (merged_df['Destination']=='WAS') &
        (merged_df['Vagon']=='Z') &
        (merged_df['DBD']==1)]

In [92]:
xx.Revenue.values

array([61., 46., 30.])